import modules

In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler,MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import scipy.stats as stats

load the data sets

In [ ]:
df_wells=pd.read_csv(r"C:\\Users\User\\OneDrive - COG KENYA\\Documents\\PERSONAL INFO\\ML SPE\\spe_africa_dseats_datathon_2025_wells_dataset.csv",dtype=str)
df_reservoir=pd.read_csv(r"C:\\Users\\User\\OneDrive - COG KENYA\\Documents\\PERSONAL INFO\\ML SPE\\reservoir_info.csv", dtype=str)
df_classification=pd.read_csv(r"C:\\Users\\User\\OneDrive - COG KENYA\\Documents\\PERSONAL INFO\\ML SPE\\classification_parameters.csv")

Data Exploration

In [ ]:
#1. Wells Production Dataset
#df_wells.head() #get the first five rows of the dataset
#df_wells.info() # Get summaryy information about the wells data set i.e. parameters and data types
#convert object columns into float and datetime into date
df_wells['PROD_DATE']=pd.to_datetime(df_wells['PROD_DATE'],format='mixed',dayfirst=True)
for col in df_wells.columns[2:]:
    df_wells[col] = (df_wells[col].astype(str).str.replace(',', '', regex=False).astype(float))
#Get a summary of the well production dataset.
"""df_wells.groupby("WELL_NAME").agg(bhp_max=("BOTTOMHOLE_FLOWING_PRESSURE (PSI)", "max"),
        bhp_min=("BOTTOMHOLE_FLOWING_PRESSURE (PSI)", "min"),
        whp_avg=("WELL_HEAD_PRESSURE (PSI)", "mean"),
        wht_avg=("WELL_HEAD_TEMPERATURE (deg F)", "mean"),
        annulus_press_avg=("ANNULUS_PRESS (PSI)", "mean"),
        choke_size_avg=("CHOKE_SIZE (%)","mean"),
        prod_start_date=("PROD_DATE", "min"),
        prod_end_date=("PROD_DATE", "max"),
        cum_oil_prod=("CUMULATIVE_OIL_PROD (STB)", "max"),
        cum_gas_prod=("CUMULATIVE_TOTAL_GAS_PROD (MSCF)", "max"),
        cum_water_cut_prod=("CUMULATIVE_WATER_PROD (BBL)", "max"),).sort_values(by='whp_avg', ascending=False)"""

#Evaluate days where onstream hours are zero.
df_wells.loc[df_wells['ON_STREAM_HRS']==0, 'WELL_NAME'].value_counts()
#remove data where onstream hours are zero.
df_wells_eff = df_wells[df_wells['ON_STREAM_HRS'] > 0]
#check if they have been removed
(df_wells_eff['ON_STREAM_HRS'] == 0).sum()
#evaluate days were cumulative oil production is zero
df_wells_eff.loc[df_wells_eff['CUMULATIVE_OIL_PROD (STB)']==0, 'WELL_NAME'].value_counts()
#remove days where cumulative oil production is zero for well 6
df_wells_eff = df_wells_eff[df_wells_eff['CUMULATIVE_OIL_PROD (STB)'] > 0]

In [ ]:
#Evaluate reservoir data
#convert object type into numerical values
for col in df_reservoir.columns[1:]:
    df_reservoir[col] = (df_reservoir[col].astype(str).str.replace(',', '', regex=False).astype(float))
df_reservoir.info()
df_reservoir.head()

Data Visualizations
Visualizations will include production plots for oil gas and water, boxplots and correlation plots between wells.

In [ ]:
"""def plot_visualizations(df, well_name):

    well_data = df[df['WELL_NAME'] == well_name].sort_values(by='PROD_DATE')
    # Scale the cumulative production columns
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(well_data[['CUMULATIVE_OIL_PROD (STB)', 
                                                        'CUMULATIVE_FORMATION_GAS_PROD (MSCF)', 
                                                        'CUMULATIVE_WATER_PROD (BBL)']])
    scaled_df = pd.DataFrame(scaled_values, columns=['Oil', 'Gas', 'Water'], index=well_data['PROD_DATE'])
    plt.figure(figsize=(12,6))
    plt.plot(scaled_df.index, scaled_df['Oil'], label='Scaled Oil production', color='blue')
    plt.plot(scaled_df.index, scaled_df['Gas'], label='Scaled Gas production', color='green')
    plt.plot(scaled_df.index, scaled_df['Water'], label='Scaled Water production', color='orange')
    plt.xlabel('Production Date')
    plt.ylabel('Scaled Cumulative Production (0-1)')
    plt.title(f'Scaled Cumulative Production for {well_name}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
wells=df_wells_eff['WELL_NAME'].unique()
# Plot cumulative production
plot_visualizations(df_wells_eff,'Well_#14')
#for well in wells:
    #plot_visualizations(df_wells_eff,well)

#2. Production plot for all the wells
df_wells_eff['DAYS_SINCE_START'] = df_wells_eff.groupby('WELL_NAME')['PROD_DATE'].transform(lambda x: (x - x.min()).dt.days)
plt.figure(figsize=(12,6))
sns.lineplot(data=df_wells_eff, x='DAYS_SINCE_START', y='CUMULATIVE_OIL_PROD (STB)', hue='WELL_NAME', palette='tab20')
plt.xlabel('Days Since Production Start')
plt.ylabel('CUMULATIVE_OIL_PROD (STB)')
plt.title('Normalized Production Trends by Well')
plt.grid(True)
plt.tight_layout()
plt.show()

#3.Boxplotfor BHP
plt.figure(figsize=(12,6))
sns.boxplot(x='WELL_NAME', y='BOTTOMHOLE_FLOWING_PRESSURE (PSI)', data=df_wells_eff, palette='Set2',hue='WELL_NAME')
plt.xlabel('Well Name')
plt.ylabel('Bottomhole Flowing Pressure (PSI)')
plt.title('Distribution of Bottomhole Pressure by Well')
plt.xticks(rotation=90)  # rotate well names for readability
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()
"""
#4. BHP Correlation between wells
plt.figure(figsize=(12,6))
sns.ecdfplot(data=df_wells_eff, x='BOTTOMHOLE_FLOWING_PRESSURE (PSI)',hue='WELL_NAME', palette='tab20')
plt.xlabel('Bottomhole Flowing Pressure (PSI)')
plt.ylabel('Cumulative probability of observations')
plt.title('Empirical Cumulative Distribution Plot')
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

#5. Well to well correlation heatmap based on production and bottomhole pressure.

features=(df_wells_eff.groupby("WELL_NAME").agg(bhp_max=("BOTTOMHOLE_FLOWING_PRESSURE (PSI)", "max"),
        bhp_min=("BOTTOMHOLE_FLOWING_PRESSURE (PSI)", "min"),
        whp_avg=("WELL_HEAD_PRESSURE (PSI)", "mean"),
        wht_avg=("WELL_HEAD_TEMPERATURE (deg F)", "mean"),
        annulus_press_avg=("ANNULUS_PRESS (PSI)", "mean"),
        choke_size_avg=("CHOKE_SIZE (%)","mean"),
        cum_oil_prod=("CUMULATIVE_OIL_PROD (STB)", "max"),
        cum_gas_prod=("CUMULATIVE_TOTAL_GAS_PROD (MSCF)", "max"),
        cum_water_cut_prod=("CUMULATIVE_WATER_PROD (BBL)", "max")))
corr = features.T.corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Well-to-Well Correlation based on key features')
plt.show()

#6. Well similarity denogram
from scipy.cluster.hierarchy import dendrogram, linkage

Z = linkage(features, method='ward')

plt.figure(figsize=(10,6))
dendrogram(Z, labels=features.index, leaf_rotation=90)
plt.ylabel('Dissimilarity index') 
plt.xlabel('Wells')
plt.title('Well Similarity Dendrogram')
plt.show()

Feature engineering
This will include computing new features such as daily oil production, daily gas production, daily water production, formation gas oil ration 
and the watercut

In [ ]:

#1.Compute new features
def compute_new_metrics(df):
    
     
    #sort the wells by well name and production date
    df.sort_values(['WELL_NAME', 'PROD_DATE'], inplace=True)
    
        
    #Compute daily production
    
    #1. Oil production
    df['OIL_PRODUCTION'] = df.groupby('WELL_NAME')['CUMULATIVE_OIL_PROD (STB)'].diff() #compute daily production
    df.loc[df.groupby('WELL_NAME').head(1).index, 'OIL_PRODUCTION'] = df['CUMULATIVE_OIL_PROD (STB)']
    df['AVERAGE_PRODUCTION_RATE'] = np.where(df['ON_STREAM_HRS'] != 0,df['OIL_PRODUCTION']/ df['ON_STREAM_HRS'],0 )

    

    #2. Gas Production
    df['GAS_PRODUCTION'] = df.groupby('WELL_NAME')['CUMULATIVE_FORMATION_GAS_PROD (MSCF)'].diff()
    df.loc[df.groupby('WELL_NAME').head(1).index, 'GAS_PRODUCTION'] = df['CUMULATIVE_FORMATION_GAS_PROD (MSCF)']
    
    #3. Total Gas
    df['TOTAL_GAS_PRODUCTION'] = df.groupby('WELL_NAME')['CUMULATIVE_TOTAL_GAS_PROD (MSCF)'].diff()
    df.loc[df.groupby('WELL_NAME').head(1).index, 'TOTAL_GAS_PRODUCTION'] = df['CUMULATIVE_TOTAL_GAS_PROD (MSCF)']
        
    #3. Water PRODUCTION
    df['WATER_PRODUCTION'] = df.groupby('WELL_NAME')['CUMULATIVE_WATER_PROD (BBL)'].diff()
    df.loc[df.groupby('WELL_NAME').head(1).index, 'WATER_PRODUCTION'] = df['CUMULATIVE_WATER_PROD (BBL)']
        
    #4. Water cut
    df['WATER_CUT_%'] = (df['WATER_PRODUCTION'] / (df['OIL_PRODUCTION'] + df['WATER_PRODUCTION']).replace(0, np.nan) * 100).fillna(0)

    # 5.Calulate Formation Gas Oil Ratio (GOR)
    
    df['FORMATION_GOR'] = (df['GAS_PRODUCTION'] * 1000) / df['OIL_PRODUCTION']
  
    # Handle other infinite or NaN values just in case
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)
    
        
    return df


1) Identify the reservoir each well is producing from assuming not more than 200 psi 
differential pressure at the maximum bottom hole pressure

In [ ]:
#Identify the reservoir each well is producing from assuming not more than 200 psi differential pressure at the maximum bottom hole pressure.
#1.Empirical Analsysis
def assign_well_to_reservoirs(df,df_reservoir):
     """
    Assigns wells to reservoir based on psi differntial pressure at the maximum bottomhole pressure
    We use average reservoir pressure.
    
    Args:
        df (pd.DataFrame): Well production data
        df_reservoir (pd.DataFrame): reservoir data
    """
    
    #Create a dataframe for maximum bottomhole pressure
    df_ANALYSIS_DATA=df.groupby('WELL_NAME')['BOTTOMHOLE_FLOWING_PRESSURE (PSI)'].max().reset_index()
    #rename the bottmohole pressure column to bottomhole presssure
    df_ANALYSIS_DATA=df_ANALYSIS_DATA.rename(columns={'BOTTOMHOLE_FLOWING_PRESSURE (PSI)':'MAX_BOTTOMHOLE_PRESSURE'})
   
    # Initialize new columns
    df_ANALYSIS_DATA['Reservoir Name'] = None
    df_ANALYSIS_DATA['DIFFERENTIAL_PRESSURE'] = None

    for idx, row in df_ANALYSIS_DATA.iterrows():
        BHP = row['MAX_BOTTOMHOLE_PRESSURE']
        
        df_reservoir_copy = df_reservoir.copy()
        df_reservoir_copy['DIFF'] = df_reservoir_copy['Current Average Reservoir Pressure (PSI)'] - BHP
        
        valid = df_reservoir_copy[(df_reservoir_copy['DIFF'] > 0) & (df_reservoir_copy['DIFF'] <= 200)]
        
        if not valid.empty:
            best_match = valid.sort_values(by='DIFF').iloc[0]
            # Assign to existing dataframe
            df_ANALYSIS_DATA.at[idx, 'Reservoir Name'] = best_match['Reservoir Name']
            df_ANALYSIS_DATA.at[idx, 'DIFFERENTIAL_PRESSURE'] = best_match['DIFF']

    return df_ANALYSIS_DATA
df_ANALYSIS_DATA= assign_well_to_reservoirs(df_wells,df_reservoir)

2) Determine for each reservoir whether saturated or undersaturated using the initial 
condition 

In [ ]:

def evaluate_well_saturation(df):
    
     """
    Determines for each reservoir whether saturated or undersaturated using the initial condition 
    
    Args:
        df (pd.DataFrame): Well production data
        
    """
    initial_pressure='Initial Reservoir Pressure (PSI)'
    bubble_point_pressure='Bubble Point Pressure (PSI)')

    df['Reservoir Saturation']=df.apply(lambda row: 'Undersaturated' if row[initial_pressure] > row[bubble_point_pressure]
                                        else 'Saturated',
        axis=1)
    
    return df

df_reservoir=evaluate_well_saturation(df_reservoir)

3) Identify whether the well is naturally flowing or gas lifted 

In [ ]:
def evaluate_well_flow(df, df_analysis):
     """
    Determine whether the well is naturally flowing or gas lifted  
    
    Args:
        df (pd.DataFrame): Well production data
        df_analysis(pd.DataFrame): output dataframe
        
    """
    # Compute max cumulative formation gas and total gas per well
    max_formation_gas = df.groupby('WELL_NAME')['CUMULATIVE_FORMATION_GAS_PROD (MSCF)'].max()
    max_total_gas = df.groupby('WELL_NAME')['CUMULATIVE_TOTAL_GAS_PROD (MSCF)'].max()

    # Merge these into the df_analysis
    df_analysis = df_analysis.merge(max_formation_gas,left_on='WELL_NAME', right_index=True, how='left')
    df_analysis = df_analysis.merge(max_total_gas,left_on='WELL_NAME', right_index=True, how='left')
    
    # Classify lift type- using vectorization
    df_analysis['LIFT_TYPE'] = np.where(df_analysis['CUMULATIVE_FORMATION_GAS_PROD (MSCF)'] >= df_analysis['CUMULATIVE_TOTAL_GAS_PROD (MSCF)'],
        'Naturally Flowing','Gas Lifted')

    return df_analysis
 
df_ANALYSIS_DATA= evaluate_well_flow(df_wells, df_ANALYSIS_DATA)


In [ ]:
#compute new data metrics i.e. average oil production, formation gas rate, water cut, daily production rates.
df_wells_eff=compute_new_metrics(df_wells)

4) Determine if production from a well is steady or unsteady. Consider unsteady wells as 
those with more than 50% drop in oil production at least once every 3 to 6 months 

In [ ]:
def Production_stability(df,df_anlysis):
    """
    Vectorized rolling-window production stability check.
    Unsteady if >50% drop occurs within any 3–6 month window.
    
    """
    rate_col='AVERAGE_PRODUCTION_RATE'
    date_col='PROD_DATE'
    min_days=90
    max_days=180
    drop_threshold=0.5

    df = df.sort_values(['WELL_NAME', date_col])

    results = []

    for well, group in df.groupby('WELL_NAME'):
        dates = group[date_col].values.astype('datetime64[D]')
        rates = group[rate_col].values

        unsteady = False

        for i in range(len(group)):
            #ignore if the drops starts from zero as you cannot have a drop from zero. This handles zero error
            if rates[i] <= 0:
                continue

            # Calculated days differences from the current date
            day_diff = (dates - dates[i]).astype(int)

            #Create boolean mask for the 90-180 window
            mask = (day_diff >= min_days) & (day_diff <= max_days)

            #skip if the mask is empty
            if not np.any(mask):
                continue

            #calculates minimum production future rates within the window
            min_future_rate = np.nanmin(rates[mask])

            #calculates drop in the production rate within the window i.e. compute the difference between the current
            #and the minimum rate in the future.
            drop_fraction = (rates[i] - min_future_rate) / rates[i]

            #if the drop is greater than 50% , flag as unstable
            if drop_fraction > drop_threshold:
                unsteady = True
                break

        results.append({ 'WELL_NAME': well, 'PRODUCTION_STABILITY': 'Unsteady' if unsteady else 'Steady' })

    stability_df = pd.DataFrame(results)

    df_anlysis = df_anlysis.merge( stability_df, on='WELL_NAME', how='left')

    return df_anlysis

    
df_ANALYSIS_DATA=Production_stability(df_wells_eff, df_ANALYSIS_DATA)

5)  Determine from the formation GOR trend whether the well is flowing at/below or 
above the solution GOR of the selected reservoir or a combination of both trends 

In [ ]:
df_ANALYSIS_DATA.head(20)

In [ ]:
df_wells_eff= df_wells_eff.merge(df_ANALYSIS_DATA[['WELL_NAME','Reservoir Name','LIFT_TYPE']], on='WELL_NAME', how='left')
df_wells_eff = df_wells_eff.merge(df_reservoir[['Reservoir Name','Initial Reservoir Pressure (PSI)','Current Average Reservoir Pressure (PSI)',
                                                'Solution Gas-Oil-Ratio (SCF/BBL)','Formation Volume Factor (RB/STB)', 'Reservoir Saturation']], on='Reservoir Name', how='left')

In [ ]:
#Evaluate days where onstream hours are zero.
df_wells_eff.loc[df_wells_eff['FORMATION_GOR']==0, 'WELL_NAME'].value_counts()
#remove days where cumulative oil production is zero for well 6
df_wells_eff = df_wells_eff[df_wells_eff['FORMATION_GOR'] > 0]

In [ ]:
def plot_gor_comparison(df_well, well_name,df_reservoir):
    """
    Plots Formation GOR vs. Solution GOR for a single well.
    
    Args:
        df_well (pd.DataFrame): Well production data
        well_name (str): Name of the well to plot
    """
    # Filter data for the selected well
    
    df_well= df_well[df_well['WELL_NAME'] == well_name].copy()
    
    # Create figure
    plt.figure(figsize=(14, 6))
    
    # Plot Formation GOR (solid red line)
    ax = sns.lineplot(
        data=df_well, x='PROD_DATE', y='FORMATION_GOR',
        color='red', linewidth=2, linestyle='-',
        label='Formation GOR'
    )
    
    # Plot Solution GOR (dashed orange line)
    sns.lineplot(
        data=df_well, x='PROD_DATE', y='Solution Gas-Oil-Ratio (SCF/BBL)',
        color='orange', linewidth=2, linestyle='--',
        label='Solution GOR (Rs)'
    )
    
    # Formatting
    plt.title(f'{well_name}: Formation GOR vs. Solution GOR')
    plt.ylabel('GOR (SCF/STB)')
    plt.xlabel('Production Date')
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
for well in df_wells_eff['WELL_NAME'].unique():
     plot_gor_comparison(df_wells_eff,well,df_reservoir)

In [ ]:

def evaluate_gor_solution_trend(df,df_analysis, threshold=0.10):
    """
    Classifies wells based on dominance of GOR relative to solution GOR (Rs)
    Rules:
    aSolGOR: Above Solution GOR
    Combo: Combination of two or more trends 
    bSolGOR: At or below solution gas oil ratio (GOR)   
    """
    df = df.copy()

    # Vectorized comparisons
    df['aSolGOR'] = df['FORMATION_GOR'] > df['Solution Gas-Oil-Ratio (SCF/BBL)']
    df['bSolGOR'] = df['FORMATION_GOR'] <= df['Solution Gas-Oil-Ratio (SCF/BBL)']

    # Aggregate the data per well
    summary = (df.groupby('WELL_NAME').agg(TOTAL_POINTS=('FORMATION_GOR', 'count'),
            ABOVE_COUNT=('aSolGOR', 'sum'), BELOW_COUNT=('bSolGOR', 'sum'),
            RS=('Solution Gas-Oil-Ratio (SCF/BBL)', 'first')).reset_index())

    # compute aggregations to compare against the threashold
    summary['PCT_ABOVE'] = summary['ABOVE_COUNT'] / summary['TOTAL_POINTS']
    summary['PCT_BELOW'] = summary['BELOW_COUNT'] / summary['TOTAL_POINTS']

    #Evaluate the values against the threashold to evaluate the flow.
    summary['GOR_SOLUTION_TREND_1'] = np.select([summary['PCT_BELOW'] <= threshold,
            summary['PCT_ABOVE'] <= threshold],['aSolGOR','bSolGOR'],default='Combo')
    #create a frame with the well name and the trend
    summary = summary[['WELL_NAME', 'GOR_SOLUTION_TREND_1']]

    # Merge ther results into the analysis dataframe
    df_analysis = df_analysis.merge(summary,on='WELL_NAME', how='left')


    return df_analysis


In [ ]:
df_ANALYSIS_DATA=evaluate_gor_solution_trend(df_wells_eff,df_ANALYSIS_DATA, threshold=0.10)

6) Determine from the watercut trend whether it is Flat, Increasing, Decreasing or a Combination of 2 or more trends
This will include plotting the watercut trend for all the well. Subsequently, we will determine the slope of each trend using 90 day rolling window and identify the dorminant trend.

In [ ]:
def plot_water_trend(df_well, well_name):
    """
    Plots Watercut trend for a single well.
    
    Args:
        df_well (pd.DataFrame): Well production data
        well_name (str): Name of the well to plot
    """
    # Filter data for the selected well
    
    df_well= df_well[df_well['WELL_NAME'] == well_name].copy()
    
    # Create figure
    plt.figure(figsize=(14, 6))
    
    # Plot Formation GOR (solid red line)
    ax = sns.lineplot(data=df_well, x='PROD_DATE', y='WATER_CUT_%', color='red', 
                      linewidth=2, linestyle='-',label='Water Cut' )
    
    # Formatting
    plt.title(f'{well_name}: Water Cut Trend')
    plt.ylabel('BBL')
    plt.xlabel('Production Date')
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
#plot water cut trend for all the wells
for well in df_wells_eff['WELL_NAME'].unique():
    plot_water_trend(df_wells_eff,well)

In [ ]:
#Evaluate the water cut trend
def determine_watercut_trend(df,df_analysis):
    """
    The function computes slope for trend over 90 days rolling window
    and evaluates the dorminant trend
     Args:
        df (pd.DataFrame): Well production data
        df_analysis (pd.DataFrame): dataframe to store output
       
       """
   
    window_days=90 #determine slope on 90 days rolling window
    min_points=5 # atleast have a minimum of 5 points per window
    slope_tol=0.05 #minimum slope tolerance for trend to be considered flat
    results = []

    for well, group in df.groupby('WELL_NAME'):
        group = group.sort_values('PROD_DATE').reset_index(drop=True)

        # Convert time to days since start
        group['DAYS'] = (group['PROD_DATE'] - group['PROD_DATE'].min()).dt.days
        days = group['DAYS'].values
        wc = group['WATER_CUT_%'].values

        trend_labels = []

        for i in range(len(group)):
            # rolling window mask
            mask = (days >= days[i]) & (days <= days[i] + window_days)

            if mask.sum() < min_points:
                continue

            X = days[mask].reshape(-1, 1)
            y = wc[mask]

            model = LinearRegression().fit(X, y)
            slope = model.coef_[0]

            if abs(slope) < slope_tol:
                trend_labels.append('Flat')
            elif slope > 0:
                trend_labels.append('Incr')
            else:
                trend_labels.append('Decr')

        # Determine dominant trend
        if not trend_labels:
            overall_trend = 'Insufficient Data'
        else:
            dominant = pd.Series(trend_labels).value_counts(normalize=True)

            if dominant.iloc[0] >= 0.6:
                overall_trend = dominant.index[0]
            else:
                overall_trend = 'Combo'

        results.append({
            'WELL_NAME': well,
            'WATERCUT_TREND': overall_trend
        })

    df_water_trend = pd.DataFrame(results)

    # Merge into analysis dataframe
    df_analysis = df_analysis.merge(
        df_water_trend,
        on='WELL_NAME',
        how='left'
    )

    return df_analysis

    

       

In [ ]:
df_ANALYSIS_DATA=determine_watercut_trend(df_wells_eff, df_ANALYSIS_DATA)

In [ ]:
df_ANALYSIS_DATA.head(20)

7) Determine from the oil productivity index trend whether it is Flat, Increasing, Decreasing or a Combination of 2 or more trends. You may use average reservoir pressure for the oil productivity index computation. We will start by computing the Oil productivity index using the formula :PI=Q/(Pr-Pwf)) where PI is Productivity Index , Q is the Oil flow rate ,Pr is the  Average Reservoir Pressure and Pwf is the Bottom-hole Flowing Pressure.
Then we will plot the productivity index and evaluate the trend.

In [ ]:

#Calculate the Oil productivity index
df_wells_eff['OIL_PRODUCTIVITY_INDEX']=(df_wells_eff['AVERAGE_PRODUCTION_RATE']/(df_wells_eff['Current Average Reservoir Pressure (PSI)']-df_wells_eff['BOTTOMHOLE_FLOWING_PRESSURE (PSI)']))*24

In [ ]:
#Evaluate days where onstream hours are zero.
df_wells_eff.loc[df_wells_eff['OIL_PRODUCTIVITY_INDEX']==0, 'WELL_NAME'].value_counts()
#remove days where cumulative oil production is zero for well 6
df_wells_eff = df_wells_eff[df_wells_eff['OIL_PRODUCTIVITY_INDEX'] > 0]

In [ ]:
#plot for oil productivity index
def plot_oil_productivity_index(df_well, well_name):
    """
    Plots oil productivity index for a single well.
    
    Args:
        df_well (pd.DataFrame): Well production data
        well_name (str): Name of the well to plot
    """
    # Filter data for the selected well
    
    df_well= df_well[df_well['WELL_NAME'] == well_name].copy()
    
    # Create figure
    plt.figure(figsize=(14, 6))
    
    # Plot Formation GOR (solid red line)
    ax = sns.lineplot(data=df_well, x='PROD_DATE', y='OIL_PRODUCTIVITY_INDEX', color='red', 
                      linewidth=2, linestyle='-',label='Oil productivity index' )
    
    # Formatting
    plt.title(f'{well_name}: Oil productivity index')
    plt.ylabel('Productivity index')
    plt.xlabel('Production Date')
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.tight_layout()
    plt.show()

In [ ]:
for well in df_wells_eff['WELL_NAME'].unique():
    plot_oil_productivity_index(df_wells_eff,well)

In [ ]:
#oil productivity index:

def determine_oil_productivity_trend(df,df_analysis):
    """
    The function computes slope for trend over 90 days rolling window
    and evaluates the dorminant trend
    Args:
        df (pd.DataFrame): Well production data
        df_analysis (pd.DataFrame): dataframe to store output
       
    """
    window_days=180 #determine slope on 90 days rolling window
    min_points=5 # atleast have a minimum of 5 points per window
    slope_tol=0.001#minimum slope tolerance for trend to be considered flat
    results = []

    for well, group in df.groupby('WELL_NAME'):
        group = group.sort_values('PROD_DATE').reset_index(drop=True)

        # Convert time to days since start
        group['DAYS'] = (group['PROD_DATE'] - group['PROD_DATE'].min()).dt.days
        days = group['DAYS'].values
        wc = group['OIL_PRODUCTIVITY_INDEX'].values

        trend_labels = []

        for i in range(len(group)):
            # rolling window mask
            mask = (days >= days[i]) & (days <= days[i] + window_days)

            if mask.sum() < min_points:
                continue

            X = days[mask].reshape(-1, 1)
            y = wc[mask]

            model = LinearRegression().fit(X, y)
            slope = model.coef_[0]

            if abs(slope) < slope_tol:
                trend_labels.append('Flat')
            elif slope > 0:
                trend_labels.append('Incr')
            else:
                trend_labels.append('Decr')

        # Determine dominant trend
        if not trend_labels:
            overall_trend = 'Insufficient Data'
        else:
            dominant = pd.Series(trend_labels).value_counts(normalize=True)

            if dominant.iloc[0] >= 0.6:
                overall_trend = dominant.index[0]
            else:
                overall_trend = 'Combo'

        results.append({
            'WELL_NAME': well,
            'OIL_PRODUCTIVITY_INDEX_TREND6': overall_trend
        })

    df_water_trend = pd.DataFrame(results)

    # Merge into analysis dataframe
    df_analysis = df_analysis.merge(
        df_water_trend,
        on='WELL_NAME',
        how='left'
    )

    return df_analysis

In [ ]:
df_ANALYSIS_DATA=determine_oil_productivity_trend(df_wells_eff, df_ANALYSIS_DATA)

In [ ]:
df_ANALYSIS_DATA.head(20)